In [2]:
import os 

In [3]:
!pip install tensorflow 

In [4]:
import tensorflow as tf 
from tensorflow import keras 


2024-11-15 22:34:42.130495: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 22:34:43.516825: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 22:34:46.200666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import cv2 


data = []
categories = ['with_mask','without_mask']



for category in categories:
    path = os.path.join('train', category)
    label = os.path.join('train', category)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224, 224))
        data.append([img, label])




In [6]:
len(data)

7553

In [7]:
import random 
random.shuffle(data)

In [8]:

x = []
y = []

for feature, label in data:
    x.append(feature)
    y.append(label)

print(len(x))
print(len(y))

7553
7553


In [9]:
import numpy as np 
x = np.array(x)
y = np.array(y)

y.shape 

(7553,)

In [10]:
x.shape 

(7553, 224, 224, 3)

In [12]:
# x = x/255 
y = y.astype(np.int32)



ValueError: invalid literal for int() with base 10: 'train/with_mask'

In [ ]:
from sklearn.model_selection import train_test_split 

xtrain, xtest, ytrain, ytest  = train_test_split(x,y, test_size = 0.2)


In [ ]:
xtrain.shape 

(6042, 224, 224, 3)

In [ ]:
xtest.shape 

(1511, 224, 224, 3)

In [13]:
data = []
categories = ['with_mask', 'without_mask']

for category in categories:
    path = os.path.join('train', category)
    
    # Set label as 0 or 1 for binary classification
    label = 0 if category == 'without_mask' else 1
    
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize to match VGG16 input size
        data.append([img, label])

# Shuffle and split into features (x) and labels (y)
import random
random.shuffle(data)

x = []
y = []

for feature, label in data:
    x.append(feature)
    y.append(label)

x = np.array(x)
y = np.array(y)

# Normalize images to [0, 1] range
# x = x / 255.0

# Train-test split
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)



In [14]:
from keras.applications.vgg16 import VGG16 
vgg = VGG16()

2024-11-15 22:40:34.673299: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-11-15 22:40:34.962349: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-11-15 22:40:35.153902: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-11-15 22:40:35.552503: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-11-15 22:40:35.600918: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


In [15]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
from keras import Sequential 



In [17]:
model = Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [19]:
for layer in model.layers:
    layer.trainable = False 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [20]:
from keras.layers import Dense 
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [21]:
# from tensorflow.keras.losses import BinaryCrossentropy

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(xtrain, ytrain, epochs=1, validation_data =(xtest, ytest)) 


189/189 [==============================] - 3970s 21s/step - loss: 0.0629 - accuracy: 0.9750 - val_loss: 0.0333 - val_accuracy: 0.9868


In [22]:
model.save('mask_detector.h5')


/home/rohan/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
model.predict('train/without_mask/without_mask_1.jpg')

IndexError: tuple index out of range